<a href="https://colab.research.google.com/github/rixprakash/linearRegression/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/rixprakash/linearRegression

Cloning into 'linearRegression'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 69 (delta 22), reused 11 (delta 11), pack-reused 37 (from 1)
Receiving objects: 100% (69/69), 2.64 MiB | 6.41 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Assignment: Linear Models
## Foundations of Machine Learning
## Do Q1 and one other question

**Q0.** Please answer the following questions in your own words.

1. What makes a model "linear"? "Linear" in what?

A model is linear if the relationship between the dependent variable and the independent variables are linear. The model predictions are formed as a linear combination of the parameters, so this relationships doesn't need to be a straight line but rather it is just all to the first power and no exponents.

2. How do you interpret the coefficient for a dummy/one-hot-encoded variable? (This is a trick question, and the trick involves how you handle the intercept of the model.)

This depends on how the intercept is handled in the model. The coeefficient of the dummy varibales tells you the difference in the dependent variable when the dummy variable category is present relative to the base category (dummy is 0). If there is no intercept though, each category represented by the dummy variable is interpreted independently.

3. Can linear regression be used for classification? Explain why, or why not.

It is not a good choice for classification. This is because linear regression predicts continous values and the issues with interpretability and boundary issues for classification purposes.

4. What are signs that your linear model is over-fitting?

Overfitting is when the model is too complex and captures all the noise instead of the real pattern. Some signs include: high training accuracy and low test accuracy, low residual errors on training set but high error on validation data, complex models with many predictors (complex terms too like interactions).

5. Clearly explain multi-colinearity using the two-stage least squares technique.

Multicollinearity occurs when 2 or more predictor variables are highly correlated which makes it hard to estimate their effects on the dependent variable. With the two stage least squares technique, multicollinearity can occur between the instruments and the endogenous variables (endogenous is the correlation between the IV and the error term).

6. What are two ways to incorporate nonlinear relationships between your target/response/dependent/outcome variable $y$ and your features/control/response/independent variables $x$?

One way is incorporating polynomial terms to the linear model to allow for the model to fit curved relationships between x and y. Another way is transform dependent or independent variables to capture nonlinear effects such as a with logarithmic or exponential transformation.

7. What is the interpretation of the intercept? A slope coefficient for a variable? The coefficient for a dummy/one-hot-encoded variable?

The intercept is the expected value of the dependent variable when all independent variables are zero. THe slope coefficient represents the change in y for a one unit increase in x1 if all other variables are held constant. The coefficient of a dummy variable represents the difference in the outcome between the category of the dummy variable and the reference category which is the baaseline.

**Q1.** Load `./data/Q1_clean.csv`. The data include

- `Price` per night
- `Review Scores Rating`: The average rating for the property
- `Neighbourhood `: The bourough of NYC. Note the space, or rename the variable.
- `Property Type`: The kind of dwelling
- `Room Type`: The kind of space being rented

1. Compute the average prices and scores by `Neighbourhood `; which bourough is the most expensive on average? Create a kernel density plot of price and log price, grouping by `Neighbourhood `.
2. Regress price on `Neighbourhood ` by creating the appropriate dummy/one-hot-encoded variables, without an intercept in the linear model and using all the data. Compare the coefficients in the regression to the table from part 1. What pattern do you see? What are the coefficients in a regression of a continuous variable on one categorical variable?
3. Repeat part 2, but leave an intercept in the linear model. How do you have to handle the creation of the dummies differently? What is the intercept? Interpret the coefficients. How can I get the coefficients in part 2 from these new coefficients?
4. Split the sample 80/20 into a training and a test set. Run a regression of `Price` on `Review Scores Rating` and `Neighbourhood `. What is the $R^2$ and RMSE on the test set? What is the coefficient on `Review Scores Rating`? What is the most expensive kind of property you can rent?
5. Split the sample 80/20 into a training and a test set. Run a regression of `Price` on `Review Scores Rating` and `Neighbourhood ` and `Property Type`. What is the $R^2$ and RMSE on the test set? What is the coefficient on `Review Scores Rating`? What is the most expensive kind of property you can rent?
6. What does the coefficient on `Review Scores Rating` mean if it changes from part 4 to 5? Hint: Think about how multilple linear regression works.
7. (Optional) We've included `Neighborhood ` and `Property Type` separately in the model. How do you interact them, so you can have "A bedroom in Queens" or "A townhouse in Manhattan". Split the sample 80/20 into a training and a test set and run a regression including that kind of "property type X neighborhood" dummy, plus `Review Scores Rating`. How does the slope coefficient for `Review Scores Rating`, the $R^2$, and the RMSE change? Do they increase significantly compares to part 5? Are the coefficients in this regression just the sum of the coefficients for `Neighbourhood ` and `Property Type` from 5? What is the most expensive kind of property you can rent?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv('/content/linearRegression/data/Q1_clean.csv')
df.head()

,Price,Review Scores Rating,Neighbourhood,Property Type,Room Type
0,549,96.0,Manhattan,Apartment,Private room
1,149,100.0,Brooklyn,Apartment,Entire home/apt
2,250,100.0,Manhattan,Apartment,Entire home/apt
3,90,94.0,Brooklyn,Apartment,Private room
4,270,90.0,Manhattan,Apartment,Entire home/apt


In [3]:
df.loc[:,['Price','Neighbourhood '] ].groupby('Neighbourhood ').describe()

Price                                                      \
                  count        mean         std   min     25%    50%    75%   
Neighbourhood                                                                 
Bronx             217.0   75.276498   39.755468  10.0   50.00   60.0   90.0   
Brooklyn         8487.0  127.747378  106.038466  20.0   75.00  100.0  150.0   
Manhattan       11763.0  183.664286  170.434606  25.0  103.00  150.0  214.0   
Queens           1590.0   96.857233   61.712648  25.0   60.00   80.0  115.0   
Staten Island      96.0  146.166667  508.462029  35.0   54.75   71.0   99.0   

                         
                    max  
Neighbourhood            
Bronx             244.0  
Brooklyn         4500.0  
Manhattan       10000.0  
Queens            950.0  
Staten Island    5000.0

Manhattan is the most expensive bourough with an average cost of $183.66 a night.

In [ ]:
sns.kdeplot(x=df['Price'], hue=df['Neighbourhood '])
plt.show()

sns.kdeplot(x=np.log(df['Price']), hue=df['Neighbourhood '])
plt.show()